In [64]:
import api_exp
import itertools
import numpy as np
import pandas as pd
import cassiopeia
import env
import requests
import time
import os
import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
match = api_exp.main('NA1_4169229565')

['NA1_4169245913']


KeyboardInterrupt: 

In [ ]:
match.game_history_dict

In [ ]:
df = match.player_stats

In [ ]:
df.time_since_last_game = round(df.time_since_last_game)

In [ ]:
df['is_session'] = df.time_since_last_game < 46
df

In [ ]:
for val in match.game_history_dict.values():
    print(len(val))

In [115]:
def get_stat_histories(df, match_obj, filename, start_id = "", game_info=[], stopped=0, start=0):
    first_loop = True
    ids = list(match_obj.game_history_dict.keys())
    #print(ids)
    
    
    cum_sum = np.cumsum([len(val) for val in match_obj.game_history_dict.values()])
    games = [val for val in match_obj.game_history_dict.values()]
    for i, num in enumerate(cum_sum):
        #print(df.shape[0]-num)
        if df.shape[0]-num < 0:
            index_num = i
            id_start = ids.index(ids[i])
            start = df.shape[0]-num
            #print(id_start)
            break
        else:
            id_start = 0
    
    if start_id:
        id_start = ids.index(start_id)
        #print(id_start)

    for puuid in ids[id_start:]:
        #print(ids[id_start])
        if first_loop:
            counter = start
        else:
            counter = 0
        #print(counter)
        for i, game in enumerate(match_obj.game_history_dict[puuid][counter:]):
            
            if game != '0' and game != 0:
                r = requests.get(f'https://americas.api.riotgames.com/lol/match/v5/matches/{game}', headers=env.headers)
                response = r.json()
                time.sleep(1.125)
                game_duration = response['info']['gameDuration']
                if game_duration /10_000 > 1:
                    game_duration /= 1_000

                try:
                    for participant in response['info']['participants']:
                        #print(response['info']['participants'])

                        if participant['puuid'] == puuid:  
                            try:
                                game_info.append({
                                    'puuid': puuid,
                                    'summoner_name': match_obj.puuid_to_summoner[puuid],
                                    'game_id': game,
                                    'champion_id': participant['championId'],
                                    'champion_name': participant['championName'],
                                    'team_pos': participant['teamPosition'],
                                    'win': participant['win'],
                                    'game_start': response['info']['gameStartTimestamp'],
                                    'game_end': response['info']['gameStartTimestamp']+game_duration,
                                    'game_duration': game_duration,
                                    'team': participant['teamId']
                                })
                            except:
                                try:
                                    game_info.append({
                                        'puuid': puuid,
                                        'summoner_name': match_obj.puuid_to_summoner[puuid],
                                        'game_id': game,
                                        'champion_id': participant['championId'],
                                        'champion_name': participant['championName'],
                                        'team_pos': participant['teamPosition'],
                                        'win': participant['win'],
                                        'game_start': response['info']['gameStartTimestamp'],
                                        'game_end': response['info']['gameEndedInSurrender'],
                                        'game_duration': game_duration,
                                        'team': participant['teamId']
                                    })
                                except:
                                    game_info.append({
                                        'puuid': puuid,
                                        'summoner_name': match_obj.puuid_to_summoner[puuid],
                                        'game_id': game,
                                        'champion_id': participant['championId'],
                                        'champion_name': participant['championName'],
                                        'team_pos': participant['teamPosition'],
                                        'win': participant['win'],
                                        'game_start': response['info']['gameStartTimestamp'],
                                        'game_end': response['info']['gameStartTimestamp'] + game_duration, 
                                        'game_duration': game_duration,
                                        'team': participant['teamId']
                                    })

                            first_loop = False
                            df = pd.concat([df, pd.DataFrame(game_info)])
                            df = df.drop_duplicates()
                            df.to_csv(filename, index=False)

                    counter+= 1
                    print(f'\r{counter} of {len(match_obj.game_history_dict[puuid])} complete\
                    current_id :{puuid}, response: {r.status_code}', end='')

                except KeyError:
                    print(response)
                    stopped+=1
                    if stopped==2:
                        print('stopped')
                        df = pd.concat([df, pd.DataFrame(game_info)])
                        df = df.drop_duplicates()
                        df.to_csv(filename, index=False)
                        return df
                    print(f'we stopped at {counter}')
                    time.sleep(120)
                    if counter == len(game_info):
                        print(f'We can start over at index {counter}')
                    if counter == len(game_info) -1:
                        print(f'Start over at {counter}')
                    df = pd.concat([df, pd.DataFrame(game_info)])
                    df = df.drop_duplicates()
                    df.to_csv(filename, index=False)
                    
                    get_stat_histories(df, match_obj, filename, start_id=puuid, game_info=game_info, stopped=stopped, start=i)
                
                except KeyboardInterrupt:
                    return df
                

    return pd.DataFrame(game_info)

In [ ]:

df = pd.read_csv('checkpoint_stats.csv')
df = get_stat_histories(df, match, filename)


In [ ]:
df = pd.read_csv('checkpoint_stats.csv')
df.shape

In [ ]:
df.head()

In [ ]:
match2 = api_exp.main('NA1_4169245913')

In [116]:
def lobby_and_player_stats(match_id):
    lobby = f'riot_api_data/{match_id}_lobby.csv'
    histories = f'riot_api_data/{match_id}_histories.csv'

    match = api_exp.main(match_id)
    df = match.player_stats
    #df.last_100_games = df.last_100_games.astype('object')
    df['is_session'] = df.time_since_last_game /60 < 46
    
    df.to_csv(lobby, index=False) 
    df = pd.read_csv(lobby)
    #print(df)
    time.sleep(1)
    
    if os.path.isfile(histories):
        player_stats = pd.read_csv(histories)
    else:
        player_stats = pd.DataFrame()
        
    player_stats = get_stat_histories(player_stats, match, filename=histories)
        
    return df, player_stats


In [117]:
game_nums = [4169229565, 4169245913, 4035079604, 4035079613, 4035079665, 4035079666, 4035079668, 4035079686, 4035079692, 4035079696, 4035079700, 4035079718, 4035079733, 4035089527, 4035089567, 4035089568, 4035089573, 4035089579, 4035089582, 4035089606, 4035089614, 4035089624, 4035089626, 4035089632, 4035089659, 4035089661, 4035089662, 4035089666, 4035089690, 4035089707, 4035089711, 4035089723, 4035089726, 4035098534, 4035098537, 4035098619, 4035098637, 4035098655, 4035098658, 4035098660, 4035098675, 4035098702, 4035098709, 4035098711, 4035098724, 4035098732, 4035098733, 4035098734, 4035098778, 4035098785, 4035098786, 4035098799, 4035098805, 4035098818, 4035098855, 4035098860, 4035098870, 4035098873, 4035098879, 4035098884, 4035098890, 4035098893, 4035098897, 4035098898, 4035098913, 4035098925, 4035098937, 4035098938, 4035098957, 4035098967, 4035098968, 4035098977, 4035098999, 4035099000, 4035099001, 4035099011, 4035099026, 4035099035, 4035099036, 4035099037, 4035099057, 4035099061, 4035099064, 4035099065, 4035099069, 4035099089, 4035099107, 4035099109, 4035099114, 4035099129, 4035099132, 4035099133, 4035099162, 4035099172, 4035099182, 4035099192, 4035099203, 4035099220, 4035099226, 4035099230, 4035099234, 4035099239, 4035099242, 4035099246, 4035099247, 4035099256, 4035099257, 4035099262, 4035099279, 4035099329, 4035099335, 4035099363, 4035099409, 4035099421, 4035099425, 4035099437, 4035099446, 4035099447, 4035099456, 4035099458, 4035099496, 4035099508, 4035099514, 4035099519, 4035099527, 4035099540, 4035099552, 4035099561, 4035099570, 4035099578, 4035099585, 4035099593, 4035099606, 4035099619, 4035099623, 4035099626, 4035099643, 4035099652, 4035099674, 4035099694, 4035099705, 4035099710, 4035099721, 4035099731, 4035099734, 4035108167, 4035108182, 4035108202, 4035108225, 4035108251, 4035108252, 4035108255, 4035108260, 4035108265, 4035108271, 4035108291, 4035108299, 4035108311, 4035108330, 4035108345, 4035108355, 4035108366, 4035108372, 4035108405]
game_ids= ['NA1_'+str(num) for num in game_nums]
#for a_game in game_ids:
#    try:
#        lobby_and_player_stats(a_game)
#    except:
#        print(a_game)

In [ ]:
error_num = 4035089527


game_nums.index(error_num)
gamma = ['NA1_'+str(num) for num in game_nums[game_nums.index(error_num):]]
for gam in gamma:

    print('sleeping')
    print(gam)
    time.sleep(30)
    lobby_and_player_stats(gam)


sleeping
NA1_4035089527
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
['NA1_4035063794']
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
['NA1_4032560539']
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
['NA1_4035105136']
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
['NA1_4035105136']
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
['NA1_4035101901']
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
['NA1_4034060121']
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987231.0
CREATION TIME!!!: 1630987

In [80]:
datetime.datetime.fromtimestamp(1641654218)

datetime.datetime(2022, 1, 8, 9, 3, 38)

In [77]:
datetime.datetime.fromtimestamp(1641652592)

datetime.datetime(2022, 1, 8, 8, 36, 32)

In [79]:
datetime.datetime.fromtimestamp(1641650370)

datetime.datetime(2022, 1, 8, 7, 59, 30)